In [1]:
import sys

sys.path.append("../../ares_transverse_tuning")

In [2]:
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
from gymnasium.wrappers import (
    FilterObservation,
    FlattenObservation,
    RescaleAction,
    TimeLimit,
)
from src.bayesopt import BayesianOptimizationAgent
from src.environments import ea
from src.trial import Trial, load_trials
from src.wrappers import NotVecNormalize, PolishedDonkeyCompatibility, RecordEpisode
from src.xopt_helper import (
    BOXoptCompatibleWrapper,
    XoptAgent,
    prepare_ARES_EA_data,
    rescale_magnet_values,
)
from stable_baselines3 import TD3
from tqdm import tqdm
from xopt import VOCS

In [3]:
DATA_DIR = Path("../../data/paper/baselines")
TRIAL_INDICIES = [0, 33, 38]
NUM_REPEATS = 3

In [4]:
trials = load_trials(Path("../../ares_transverse_tuning/data/trials.yaml"))
experiment_indicies = TRIAL_INDICIES * NUM_REPEATS
experiment_indicies

[0, 33, 38, 0, 33, 38, 0, 33, 38]

## Reinforcement Learning


In [5]:
def try_problem_rl(trial_index: int, trial: Trial, write_data: bool = True) -> None:
    model_name = "polished-donkey-996"

    # Load the model
    model = TD3.load(
        f"../../ares_transverse_tuning/models/ea/legacy/{model_name}/model"
    )

    # Create the environment
    env = ea.TransverseTuning(
        backend="cheetah",
        backend_args={
            "incoming_mode": trial.incoming_beam,
            "misalignment_mode": trial.misalignments,
        },
        action_mode="delta",
        magnet_init_mode=np.array([10, -10, 0, 10, 0]),
        max_quad_delta=30 * 0.1,
        max_steerer_delta=6e-3 * 0.1,
        target_beam_mode=trial.target_beam,
        target_threshold=None,
        threshold_hold=5,
        clip_magnets=True,
    )
    env = TimeLimit(env, 150)
    if write_data:
        env = RecordEpisode(
            env,
            save_dir=str(
                DATA_DIR
                / "rl"
                / f"trial-{trial_index}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
            ),
        )
    env = FilterObservation(env, ["beam", "magnets", "target"])
    env = FlattenObservation(env)
    env = PolishedDonkeyCompatibility(env)
    env = NotVecNormalize(
        env,
        f"../../ares_transverse_tuning/models/ea/legacy/{model_name}/vec_normalize.pkl",
    )
    env = RescaleAction(env, -1, 1)

    # Actual optimisation
    observation, info = env.reset()
    done = False
    while not done:
        action, _ = model.predict(observation, deterministic=True)
        observation, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
    env.close()

In [6]:
for trial_index in tqdm(experiment_indicies):
    if len(list((DATA_DIR / "rl").glob(f"trial-{trial_index}_*"))) >= NUM_REPEATS:
        print(
            f"Skipping trial {trial_index} because it has already been run "
            f"{NUM_REPEATS} times."
        )
        continue

    try_problem_rl(trial_index, trials[trial_index], write_data=True)

100%|██████████| 9/9 [00:00<00:00, 4739.33it/s]

Skipping trial 0 because it has already been run 3 times.
Skipping trial 33 because it has already been run 3 times.
Skipping trial 38 because it has already been run 3 times.
Skipping trial 0 because it has already been run 3 times.
Skipping trial 33 because it has already been run 3 times.
Skipping trial 38 because it has already been run 3 times.
Skipping trial 0 because it has already been run 3 times.
Skipping trial 33 because it has already been run 3 times.
Skipping trial 38 because it has already been run 3 times.


## Bayesian Optimisation


In [7]:
def try_problem_bo(
    trial_index: int,
    trial: Trial,
    rescale_action: bool = False,
    write_data: bool = True,
) -> None:
    # Create the environment
    env = ea.TransverseTuning(
        backend="cheetah",
        backend_args={
            "incoming_mode": trial.incoming_beam,
            "misalignment_mode": trial.misalignments,
        },
        action_mode="direct",
        magnet_init_mode=np.array([10, -10, 0, 10, 0]),
        target_beam_mode=trial.target_beam,
    )
    env = TimeLimit(env, 150)
    if write_data:
        env = RecordEpisode(
            env,
            save_dir=str(
                DATA_DIR
                / "bo_hard"
                / f"trial-{trial_index}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
            ),
        )

    if rescale_action:
        env = RescaleAction(env, -1, 1)

    env = BOXoptCompatibleWrapper(env, prepare_ARES_EA_data)

    vocs = VOCS(
        variables={
            "q1": [-72, 72],
            "q2": [-72, 72],
            "cv": [-6.1782e-3, 6.1782e-3],
            "q3": [-72, 72],
            "ch": [-6.1782e-3, 6.1782e-3],
        },
        # objectives={"mae": "MINIMIZE"},
        objectives={"logmae": "MINIMIZE"},
        # constraints={"max_beamparam": ["LESS_THAN", 2e-3]},
    )

    # Tuned hyperparameters
    optimizer_kwargs = {
        "beta": 2.0,
        "max_travel_distances": [0.1] * 5,
        "proximal_weights": None,
    }

    # optimizer_kwargs = {
    #     "beta": 2.0,
    #     "proximal_weights": 0.5,
    #     "max_travel_distances": 1,
    # }

    xopt_agent = XoptAgent(
        env,
        vocs,
        method="UCB",
        action_order=["q1", "q2", "cv", "q3", "ch"],
        **optimizer_kwargs,
    )

    # Actual optimisation
    output, info = env.reset()
    if rescale_action:
        init_magnet_values = rescale_magnet_values(
            env.unwrapped.backend.get_magnets(), env.env
        )
    else:
        init_magnet_values = env.unwrapped.backend.get_magnets()
    init_sample = [
        {k: v for k, v in zip(["q1", "q2", "cv", "q3", "ch"], init_magnet_values)}
    ]
    xopt_agent.add_data(pd.DataFrame(init_sample), output)
    done = False
    while not done:
        action = xopt_agent.predict(n_samples=1)
        output, reward, done, truncated, info = env.step(action)
        xopt_agent.add_data(xopt_agent.last_sample, output)

    env.close()

In [8]:
for trial_index in tqdm(experiment_indicies):
    if len(list((DATA_DIR / "bo_hard").glob(f"trial-{trial_index}_*"))) >= NUM_REPEATS:
        print(
            f"Skipping trial {trial_index} because it has already been run "
            f"{NUM_REPEATS} times."
        )
        continue

    try_problem_bo(trial_index, trials[trial_index], write_data=True)

100%|██████████| 9/9 [00:00<00:00, 8426.06it/s]

Skipping trial 0 because it has already been run 3 times.
Skipping trial 33 because it has already been run 3 times.
Skipping trial 38 because it has already been run 3 times.
Skipping trial 0 because it has already been run 3 times.
Skipping trial 33 because it has already been run 3 times.
Skipping trial 38 because it has already been run 3 times.
Skipping trial 0 because it has already been run 3 times.
Skipping trial 33 because it has already been run 3 times.
Skipping trial 38 because it has already been run 3 times.


## Extremum Seeking


In [9]:
def try_problem_es(
    trial_index: int,
    trial: Trial,
    version="tuned",
    rescale_action: bool = True,
    write_data: bool = True,
):
    if version == "tuned":
        # Tuned hyperparameters
        optimizer_kwargs = {
            "k": 3.7,
            "oscillation_size": 0.11,
            "decay_rate": 0.987,
        }
        save_dir = f"data/bo_vs_rl/simulation/es_tuned/problem_{trial_index:03d}"
    elif version == "default":
        # Default hyperparameters
        optimizer_kwargs = {
            "k": 2.0,
            "oscillation_size": 0.1,
            "decay_rate": 1.0,
        }
        save_dir = f"data/bo_vs_rl/simulation/es_default/problem_{trial_index:03d}"
    elif version == "with_decay":
        # Default hyperparameters
        optimizer_kwargs = {
            "k": 2.0,
            "oscillation_size": 0.1,
            "decay_rate": 0.99,
        }
        save_dir = f"data/bo_vs_rl/simulation/es_with_decay/problem_{trial_index:03d}"
    else:
        raise ValueError(f"Unknown version: {version}")

    # Create the environment
    env = ea.TransverseTuning(
        backend="cheetah",
        backend_args={
            "incoming_mode": trial.incoming_beam,
            "misalignment_mode": trial.misalignments,
        },
        action_mode="direct",
        magnet_init_mode=np.array([10, -10, 0, 10, 0]),
        target_beam_mode=trial.target_beam,
    )
    env = TimeLimit(env, 150)
    if write_data:
        env = RecordEpisode(
            env,
            save_dir=str(
                DATA_DIR
                / "es"
                / f"trial-{trial_index}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
            ),
        )

    if rescale_action:
        env = RescaleAction(env, -1, 1)

    env = BOXoptCompatibleWrapper(env, prepare_ARES_EA_data)

    if rescale_action is not None:
        vocs_variables = {
            "q1": [-1, 1],
            "q2": [-1, 1],
            "cv": [-1, 1],
            "q3": [-1, 1],
            "ch": [-1, 1],
        }
    else:
        vocs_variables = {
            "q1": [-72, 72],
            "q2": [-72, 72],
            "cv": [-6.1782e-3, 6.1782e-3],
            "q3": [-72, 72],
            "ch": [-6.1782e-3, 6.1782e-3],
        }

    vocs = VOCS(
        variables=vocs_variables,
        # objectives={"mae": "MINIMIZE"},
        objectives={"logmae": "MINIMIZE"},
        # constraints={"max_beamparam": ["LESS_THAN", 2e-3]},
    )

    xopt_agent = XoptAgent(
        env,
        vocs,
        method="ES",
        action_order=["q1", "q2", "cv", "q3", "ch"],
        **optimizer_kwargs,
    )

    # Actual optimisation
    output, info = env.reset()
    if rescale_action is not None:
        init_magnet_values = rescale_magnet_values(
            env.unwrapped.backend.get_magnets(), env.env
        )
    else:
        init_magnet_values = env.unwrapped.backend.get_magnets()
    init_sample = [
        {k: v for k, v in zip(["q1", "q2", "cv", "q3", "ch"], init_magnet_values)}
    ]
    xopt_agent.add_data(pd.DataFrame(init_sample), output)
    done = False
    while not done:
        action = xopt_agent.predict(n_samples=1)
        output, reward, done, truncated, info = env.step(action)
        xopt_agent.add_data(xopt_agent.last_sample, output)

    env.close()

In [10]:
for trial_index in tqdm(experiment_indicies):
    if len(list((DATA_DIR / "es").glob(f"trial-{trial_index}_*"))) >= NUM_REPEATS:
        print(
            f"Skipping trial {trial_index} because it has already been run "
            f"{NUM_REPEATS} times."
        )
        continue

    try_problem_es(
        trial_index, trials[trial_index], version="with_decay", write_data=True
    )

100%|██████████| 9/9 [00:00<00:00, 9153.43it/s]

Skipping trial 0 because it has already been run 3 times.
Skipping trial 33 because it has already been run 3 times.
Skipping trial 38 because it has already been run 3 times.
Skipping trial 0 because it has already been run 3 times.
Skipping trial 33 because it has already been run 3 times.
Skipping trial 38 because it has already been run 3 times.
Skipping trial 0 because it has already been run 3 times.
Skipping trial 33 because it has already been run 3 times.
Skipping trial 38 because it has already been run 3 times.


## Random Search


In [11]:
def try_problem_rs(trial_index: int, trial: Trial, write_data: bool = True) -> None:
    # Create the environment
    env = ea.TransverseTuning(
        backend="cheetah",
        backend_args={
            "incoming_mode": trial.incoming_beam,
            "misalignment_mode": trial.misalignments,
        },
        action_mode="direct",
        magnet_init_mode=np.array([10, -10, 0, 10, 0]),
        target_beam_mode=trial.target_beam,
    )
    env = TimeLimit(env, 150)
    if write_data:
        env = RecordEpisode(
            env,
            save_dir=str(
                DATA_DIR
                / "rs"
                / f"trial-{trial_index}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
            ),
        )
    env = FilterObservation(env, ["beam", "magnets", "target"])
    env = FlattenObservation(env)
    env = RescaleAction(env, -1, 1)

    # Actual optimisation
    _, _ = env.reset()
    done = False
    while not done:
        action = env.action_space.sample()
        _, _, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
    env.close()

In [12]:
for trial_index in tqdm(experiment_indicies):
    if len(list((DATA_DIR / "rs").glob(f"trial-{trial_index}_*"))) >= NUM_REPEATS:
        print(
            f"Skipping trial {trial_index} because it has already been run "
            f"{NUM_REPEATS} times."
        )
        continue

    try_problem_rs(trial_index, trials[trial_index], write_data=True)

100%|██████████| 9/9 [00:00<00:00, 8573.41it/s]

Skipping trial 0 because it has already been run 3 times.
Skipping trial 33 because it has already been run 3 times.
Skipping trial 38 because it has already been run 3 times.
Skipping trial 0 because it has already been run 3 times.
Skipping trial 33 because it has already been run 3 times.
Skipping trial 38 because it has already been run 3 times.
Skipping trial 0 because it has already been run 3 times.
Skipping trial 33 because it has already been run 3 times.
Skipping trial 38 because it has already been run 3 times.


## Do Nothing


In [13]:
def try_problem_dn(trial_index: int, trial: Trial, write_data: bool = True) -> None:
    # Create the environment
    env = ea.TransverseTuning(
        backend="cheetah",
        backend_args={
            "incoming_mode": trial.incoming_beam,
            "misalignment_mode": trial.misalignments,
        },
        action_mode="direct",
        magnet_init_mode=trial.initial_magnets,
        target_beam_mode=trial.target_beam,
    )
    env = TimeLimit(env, 150)
    if write_data:
        env = RecordEpisode(
            env,
            save_dir=str(
                DATA_DIR
                / "dn"
                / f"trial-{trial_index}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
            ),
        )

    # Actual optimisation
    _, _ = env.reset()
    done = False
    while not done:
        action = trial.initial_magnets
        _, _, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
    env.close()

In [14]:
for trial_index in tqdm(experiment_indicies):
    if len(list((DATA_DIR / "dn").glob(f"trial-{trial_index}_*"))) >= NUM_REPEATS:
        print(
            f"Skipping trial {trial_index} because it has already been run "
            f"{NUM_REPEATS} times."
        )
        continue

    try_problem_dn(trial_index, trials[trial_index], write_data=True)

100%|██████████| 9/9 [00:00<00:00, 8972.84it/s]

Skipping trial 0 because it has already been run 3 times.
Skipping trial 33 because it has already been run 3 times.
Skipping trial 38 because it has already been run 3 times.
Skipping trial 0 because it has already been run 3 times.
Skipping trial 33 because it has already been run 3 times.
Skipping trial 38 because it has already been run 3 times.
Skipping trial 0 because it has already been run 3 times.
Skipping trial 33 because it has already been run 3 times.
Skipping trial 38 because it has already been run 3 times.
